In [2]:
import numpy as np
from scipy.stats import norm
from scipy.stats import uniform
from math import sqrt
import cmath #for complex numbers
from scipy.integrate import quad
import QuantLib as ql

In [10]:
def corr_brownian_motion(n, T, dim, rho):
    dt = T/n

    dW1 = norm.rvs(size=(dim,n+1) , scale=sqrt(dt))
    dW2 = rho * dW1 + np.sqrt(1 - np.power(rho ,2)) * norm.rvs(size=(dim,n+1) , scale=sqrt(dt))
        
    W1 = np.cumsum(dW1, axis=-1)
    W2 = np.cumsum(dW2, axis=-1)
 
    return W1,W2

def euler_maruyama(mu,sigma,T,x0,W):
    dim = W.shape[0]
    n = W.shape[1]-1
    Y = np.zeros((dim,n+1))
    dt = T/n
    sqrt_dt = np.sqrt(dt)
    for l in range(dim):
        Y[l,0] = x0
        for i in range(n):
            Y[l,i+1] = Y[l,i] + np.multiply(mu(Y[l,i],l,i),dt) + sigma(Y[l,i],l,i)*sqrt_dt*(W[l,i+1]-W[l,i])
    
    return Y

def milstein(mu,sigma,sigma_derivative,T,x0,W):
    dim = W.shape[0]
    n = W.shape[1]-1
    Y = np.zeros((dim,n+1))
    dt = T/n
    sqrt_dt = np.sqrt(dt)
    for l in range(dim):
        Y[l,0] = x0
        for i in range(n):
            Y[l,i+1] = Y[l,i] + mu(Y[l,i],l,i)*dt + np.multiply(sigma(Y[l,i],l,i)*sqrt_dt,(W[l,i+1]-W[l,i]))+0.5*sigma(Y[l,i],l,i)*sigma_derivative(Y[l,i],l,i)*(np.power((W[l,i+1]-W[l,i]),2)-dt)
    
    return Y

def heston(a,b,c,T,W,Z,V0,S0):
   
    #assert(2*a*b > c*c)

    def mu2(V,i,k):
        return np.multiply(a,(b-V))
    
    def sigma2(V,i,k):
        return np.multiply(c,np.sqrt(np.maximum(0.0,V)))
    
    V = euler_maruyama(mu2,sigma2,T,V0,Z)
    
    def mu1(S,i,k):
        return 0.0
    
    def sigma1(S,i,k):
        return np.multiply(np.sqrt(np.maximum(0.0,V[i,k])),S)
    
    S = euler_maruyama(mu1,sigma1,T,S0,W)
    
    return S,V

def heston_SLV(alpha,beta,a,b,c,T,W,Z,V0,S0):
   
    #assert(2*a*b > c*c)

    def mu2(V,i,k):
        return np.multiply(a,(b-V))
    
    def sigma2(V,i,k):
        return np.multiply(c,np.sqrt(np.maximum(0.0,V)))
    
    V = euler_maruyama(mu2,sigma2,T,V0,Z)
    
    def mu1(S,i,k):
        return 0.0
    
    def sigma1(S,i,k):
        return alpha*np.multiply(np.sqrt(np.maximum(0.0,V[i,k])),np.power(S,1+beta))
    
    S = euler_maruyama(mu1,sigma1,T,S0,W)
    
    return S,V

def heston_milstein(a,b,c,T,W,Z,V0,S0):
   
    #assert(2*a*b > c*c)

    def mu2(V,i,k):
        return np.multiply(a,(b-V))
    
    def sigma2(V,i,k):
        return np.multiply(c,np.sqrt(np.maximum(0.0,V)))

    def sigma_dev2(V,i,k):
        return c/2/np.sqrt(np.maximum(0.001,V))
    
    V = milstein(mu2,sigma2,sigma_dev2,T,V0,Z)
    
    def mu1(S,i,k):
        return 0.0
    
    def sigma1(S,i,k):
        return np.multiply(np.sqrt(np.maximum(0.0,V[i,k])),S)

    def sigma_dev1(S,i,k):
        return np.sqrt(np.maximum(0.0,V[i,k]))
    
    S = milstein(mu1,sigma1,sigma_dev1,T,S0,W)
    
    return S,V

def heston_SLV_milstein(alpha,beta,a,b,c,T,W,Z,V0,S0):
   
    #assert(2*a*b > c*c)

    def mu2(V,i,k):
        return np.multiply(a,(b-V))
    
    def sigma2(V,i,k):
        return np.multiply(c,np.sqrt(np.maximum(0.0,V)))

    def sigma_dev2(V,i,k):
        return c/2/np.sqrt(np.maximum(0.001,V))
    
    V = milstein(mu2,sigma2,sigma_dev2,T,V0,Z)
    
    def mu1(S,i,k):
        return 0.0
    
    def sigma1(S,i,k):
        return alpha*np.multiply(np.sqrt(np.maximum(0.0,V[i,k])),np.power(S,1+beta))

    def sigma_dev1(S,i,k):
        return alpha*(1+beta)*np.multiply(np.sqrt(np.maximum(0.0,V[i,k])),np.power(S,beta))
    
    S = euler_maruyama(mu1,sigma1,T,S0,W)
    
    return S,V

def heston_closed(a,b,c,T,K,rho,V0,S0,r=0):
    
    def char_f(w,a,b,c,T,K,rho,V0,S0,r):
        alpha = -0.5*w*(w+complex(0,1))
        beta = a - rho*c*w*complex(0,1)
        gamma = c*c/2
        h = cmath.sqrt(beta*beta-4*alpha*gamma)
        rplus = (beta + h)/c/c
        rminus = (beta - h)/c/c
        g = rminus/rplus
        D =  rminus*(1-cmath.exp(-h*T))/(1-g*cmath.exp(-h*T))
        C = a*(rminus*T-2/c/c*cmath.log((1-g*cmath.exp(-h*T))/(1-g)))
        return cmath.exp(C*b+D*V0+complex(0,1)*w*cmath.log(S0*cmath.exp(r*T)))

    def integrand1(w):
        i = complex(0,1)
        return (cmath.exp(-i*w*cmath.log(K))*char_f(w-i,a,b,c,T,K,rho,V0,S0,r)/i/w/char_f(-i,a,b,c,T,K,rho,V0,S0,r)).real
    
    def integrand2(w):
        i = complex(0,1)
        return (cmath.exp(-i*w*cmath.log(K))*char_f(w,a,b,c,T,K,rho,V0,S0,r)/i/w).real
    
    pi1 = 0.5 + quad(integrand1,0,np.inf)[0]/np.pi
    pi2 = 0.5 + quad(integrand2,0,np.inf)[0]/np.pi

    
    return (S0*pi1 + cmath.exp(-r*T)*K*pi2).real

In [7]:
def call_price(f,a,b,c,rho,T,K,V0,S0,alpha=0,beta=0):
    n = 100
    dim = 10000
    W,Z = corr_brownian_motion(n, T, dim, rho)
    if alpha==0:
        S,V = f(a,b,c,T,W,Z,V0,S0)
        S_T = S[:,n]
        P = np.mean(np.maximum(S_T-K,np.zeros(dim)))
        return P
    S,V = f(alpha,beta,a,b,c,T,W,Z,V0,S0)
    S_T = S[:,n]
    P = np.mean(np.maximum(S_T-K,np.zeros(dim)))
    return P
    

In [13]:
import time 

S0 = 1 
V0 = 0.16
a = 1
b = 0.16
c = 2
rho = -0.8
T = 10
K = 2

P1 = call_price(heston,a,b,c,rho,T,1,V0,S0)
print(P1)

P2 = call_price(f=heston_SLV,a=1,b=1,c=0.5,rho=-0.5,T=1,K=0.9,V0=0.15,S0=1,alpha=1,beta=0.0)
print(P2)

P3 = call_price(f=heston_SLV,a=1,b=1,c=0.5,rho=-0.5,T=1,K=0.9,V0=0.15,S0=1,alpha=2,beta=0.5)
print(P3)

P6 = call_price(f=heston_milstein,a=1,b=1,c=0.5,rho=-0.5,T=1,K=0.9,V0=0.15,S0=1)
print(P6)

P4 = call_price(f=heston_SLV_milstein,a=1,b=1,c=0.5,rho=-0.5,T=1,K=0.9,V0=0.15,S0=1,alpha=1,beta=0.0)
print(P4)

P5 = call_price(f=heston_SLV_milstein,a=1,b=1,c=0.5,rho=-0.5,T=1,K=0.9,V0=0.15,S0=1,alpha=2,beta=0.5)
print(P5)

0.14929650292103747
0.10129912668522278
0.11515088608316808
0.10266905312130227
0.10206566961573443
0.11607959426349641


In [12]:
P7 = heston_closed(a,b,c,T,1,rho,V0,S0,r=0)
print(P7)

1.251395849791881
